In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import math

import sys

import numpy as np

import matplotlib.pyplot as plt

import keras

import pandas as pd

import numpy as np

from keras.models import Sequential

from keras.layers import Dense

from keras.layers import LSTM

from keras.layers import Dropout

from keras.layers import *

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping

In [3]:
%cd /content/drive/MyDrive/학교/3학년2학기/소프트웨어융합개론/coin/indicator

/content/drive/MyDrive/학교/3학년2학기/소프트웨어융합개론/coin/indicator


In [4]:
df=pd.read_csv("bitcoin_indicator.csv")
df_time = pd.read_csv("time.csv")
print('Number of rows and columns:', df.shape)

time_step = 30
index_rsi = 14
n_days = 15

df

Number of rows and columns: (3962, 25)


,close,close_log,nvi,pvi,ma_5,ma_10,ma_20,ma_60,rsi,vpt,...,mfi,ema_5,ema_10,ema_20,ema_60,fi,ubb,mbb,lbb,volume
0,3091000.0,14.944005,1.163482,3063000.0,3084200.0,3170200.0,3289500.0,3.215300e+06,24.735450,-50913.192669,...,53.117928,3.101464e+06,3.154992e+06,3.207830e+06,3.195372e+06,-2.058452e+09,3.610062e+06,3289500.0,2.968938e+06,5388.617973
1,3087000.0,14.942710,1.163482,3087000.0,3088600.0,3137500.0,3279000.0,3.216817e+06,25.546448,-50921.710773,...,54.027715,3.096642e+06,3.142630e+06,3.196323e+06,3.191819e+06,-2.356487e+09,3.612042e+06,3279000.0,2.945958e+06,6582.364759
2,3071000.0,14.937514,1.157451,3087000.0,3085200.0,3106200.0,3262700.0,3.216783e+06,25.304465,-50942.230512,...,52.648713,3.088095e+06,3.129606e+06,3.184387e+06,3.187857e+06,-1.445045e+09,3.603251e+06,3262700.0,2.922149e+06,3959.027066
3,3050000.0,14.930652,1.157451,3050000.0,3072400.0,3086100.0,3243350.0,3.215633e+06,26.044568,-50979.943691,...,53.324172,3.075397e+06,3.115132e+06,3.171588e+06,3.183338e+06,-1.897196e+09,3.586173e+06,3243350.0,2.900527e+06,5515.103573
4,3003000.0,14.915122,1.139615,3050000.0,3060400.0,3063700.0,3223000.0,3.214083e+06,25.442177,-51042.834596,...,54.476339,3.051264e+06,3.094745e+06,3.155532e+06,3.177425e+06,-1.473320e+09,3.572426e+06,3223000.0,2.873574e+06,4081.218313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,22320000.0,16.920994,19.407960,22320000.0,22490200.0,22593100.0,22691850.0,2.493500e+07,46.624136,2341.093255,...,53.458386,2.245848e+07,2.256368e+07,2.277992e+07,2.431527e+07,-2.116684e+08,2.316037e+07,22691850.0,2.222333e+07,1666.680241
3958,22403000.0,16.924705,19.480131,22320000.0,22430800.0,22571300.0,22662550.0,2.482275e+07,48.006214,2346.997775,...,53.335951,2.243999e+07,2.253446e+07,2.274402e+07,2.425257e+07,-1.222620e+08,2.312610e+07,22662550.0,2.219900e+07,1587.817986
3959,23004000.0,16.951179,19.480131,23004000.0,22496800.0,22612900.0,22660250.0,2.472663e+07,51.684312,2391.934249,...,43.927284,2.262799e+07,2.261983e+07,2.276878e+07,2.421164e+07,1.172543e+08,2.311609e+07,22660250.0,2.220441e+07,1675.061266
3960,22790000.0,16.941832,19.298913,23004000.0,22581400.0,22618800.0,22651250.0,2.462907e+07,47.797063,2377.924759,...,45.856043,2.268200e+07,2.265077e+07,2.277080e+07,2.416503e+07,-1.490895e+08,2.308805e+07,22651250.0,2.221445e+07,1505.954700


In [5]:
df_time = df_time.iloc[:,0]

In [6]:
df_time = df_time.str[:10]

In [7]:
res = []

# training set/ test set

In [8]:
feature = df.iloc[:,1:]

idx = int(feature.shape[0]*0.8)

training_set = feature.iloc[:idx].values
test_set = feature.iloc[idx:].values
training_set.shape

(3169, 24)

In [9]:
training_set

array([[1.49440052e+01, 1.16348160e+00, 3.06300000e+06, ...,
        3.28950000e+06, 2.96893767e+06, 5.38861797e+03],
       [1.49427103e+01, 1.16348160e+00, 3.08700000e+06, ...,
        3.27900000e+06, 2.94595788e+06, 6.58236476e+03],
       [1.49375138e+01, 1.15745125e+00, 3.08700000e+06, ...,
        3.26270000e+06, 2.92214868e+06, 3.95902707e+03],
       ...,
       [1.81156490e+01, 2.14693282e+01, 7.37100000e+07, ...,
        7.54120500e+07, 7.09528907e+07, 2.58543345e+03],
       [1.81241328e+01, 2.16522442e+01, 7.37100000e+07, ...,
        7.52044000e+07, 7.09676991e+07, 2.22946506e+03],
       [1.81121971e+01, 2.13953462e+01, 7.37100000e+07, ...,
        7.48318500e+07, 7.14908085e+07, 1.95423116e+03]])

In [10]:
# 정규화
ss = StandardScaler()

training_set_scaled = ss.fit_transform(training_set)

X_train = []

y_train = []

for i in range(time_step, idx-n_days):
    X_train.append(training_set_scaled[i-time_step:i])

    y_train.append(training_set_scaled[i+n_days, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
y_train

array([-1.66957522, -1.66488984, -1.67506461, ...,  2.15800037,
        2.16822929,  2.15383842])

In [11]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_train.shape

(3124, 30, 24)

In [12]:
y_train.shape

(3124,)

# 모델 학습

In [13]:
model = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.4))


model.add(LSTM(100, return_sequences=True))
model.add(Dropout(0.4))

#model.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model.add(LSTM(units = 100, return_sequences = True))
model.add(Dropout(0.4))

# Adding a third LSTM layer and some Dropout regularisation

model.add(LSTM(units = 60, return_sequences = True))
model.add(Dropout(0.4))

model.add(LSTM(units = 60, return_sequences = True))
model.add(Dropout(0.4))


model.add(LSTM(units = 60, return_sequences = True))
model.add(Dropout(0.4))

# Adding a fourth LSTM layer and some Dropout regularisation

model.add(LSTM(units = 60))
model.add(Dropout(0.4))


# Adding the output layer i 
 
model.add(Dense(units = 1))

# Compiling the RNN

model.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 17s 19ms/step - loss: 0.1493
Epoch 2/80
98/98 [==============================] - 2s 19ms/step - loss: 0.0544
Epoch 3/80
98/98 [==============================] - 2s 18ms/step - loss: 0.0525
Epoch 4/80
98/98 [==============================] - 2s 19ms/step - loss: 0.0424
Epoch 5/80
98/98 [==============================] - 2s 18ms/step - loss: 0.0409
Epoch 6/80
98/98 [==============================] - 2s 19ms/step - loss: 0.0382
Epoch 7/80
98/98 [==============================] - 2s 18ms/step - loss: 0.0361
Epoch 8/80
98/98 [==============================] - 2s 18ms/step - loss: 0.0352
Epoch 9/80
98/98 [==============================] - 2s 19ms/step - loss: 0.0351
Epoch 10/80
98/98 [==============================] - 2s 19ms/step - loss: 0.0319
Epoch 11/80
98/98 [==============================] - 2s 18ms/step - loss: 0.0312
Epoch 12/80
98/98 [==============================] - 2s 19ms/step - loss: 0.0288
Epoch 13/80
98/98 [=================

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 100)           50000     
                                                                 
 lstm_1 (LSTM)               (None, 30, 100)           80400     
                                                                 
 dropout (Dropout)           (None, 30, 100)           0         
                                                                 
 lstm_2 (LSTM)               (None, 30, 100)           80400     
                                                                 
 dropout_1 (Dropout)         (None, 30, 100)           0         
                                                                 
 lstm_3 (LSTM)               (None, 30, 100)           80400     
                                                                 
 dropout_2 (Dropout)         (None, 30, 100)           0

# 모델이 예상한 값 불러오기

In [15]:
dataset_train = feature.iloc[:idx] 

dataset_test = feature.iloc[idx:]

dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)

inputs = dataset_total[len(dataset_total) - len(dataset_test) - time_step:].values

#inputs = inputs.reshape(-1,n_features)

inputs = ss.transform(inputs)

X_test = []
for i in range(time_step, dataset_test.shape[0]):

    X_test.append(inputs[i-time_step:i])

X_test = np.array(X_test)

X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], X_test.shape[2]))

print(X_test.shape)

(763, 30, 24)


In [16]:
predicted_stock_price = model.predict(X_test)

24/24 [==============================] - 3s 8ms/step


In [17]:
X_test.shape[0]

763

In [18]:
feature

,close_log,nvi,pvi,ma_5,ma_10,ma_20,ma_60,rsi,vpt,obv,...,mfi,ema_5,ema_10,ema_20,ema_60,fi,ubb,mbb,lbb,volume
0,14.944005,1.163482,3063000.0,3084200.0,3170200.0,3289500.0,3.215300e+06,24.735450,-50913.192669,7.758012e+04,...,53.117928,3.101464e+06,3.154992e+06,3.207830e+06,3.195372e+06,-2.058452e+09,3.610062e+06,3289500.0,2.968938e+06,5388.617973
1,14.942710,1.163482,3087000.0,3088600.0,3137500.0,3279000.0,3.216817e+06,25.546448,-50921.710773,7.099776e+04,...,54.027715,3.096642e+06,3.142630e+06,3.196323e+06,3.191819e+06,-2.356487e+09,3.612042e+06,3279000.0,2.945958e+06,6582.364759
2,14.937514,1.157451,3087000.0,3085200.0,3106200.0,3262700.0,3.216783e+06,25.304465,-50942.230512,6.703873e+04,...,52.648713,3.088095e+06,3.129606e+06,3.184387e+06,3.187857e+06,-1.445045e+09,3.603251e+06,3262700.0,2.922149e+06,3959.027066
3,14.930652,1.157451,3050000.0,3072400.0,3086100.0,3243350.0,3.215633e+06,26.044568,-50979.943691,6.152363e+04,...,53.324172,3.075397e+06,3.115132e+06,3.171588e+06,3.183338e+06,-1.897196e+09,3.586173e+06,3243350.0,2.900527e+06,5515.103573
4,14.915122,1.139615,3050000.0,3060400.0,3063700.0,3223000.0,3.214083e+06,25.442177,-51042.834596,5.744241e+04,...,54.476339,3.051264e+06,3.094745e+06,3.155532e+06,3.177425e+06,-1.473320e+09,3.572426e+06,3223000.0,2.873574e+06,4081.218313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3957,16.920994,19.407960,22320000.0,22490200.0,22593100.0,22691850.0,2.493500e+07,46.624136,2341.093255,1.475735e+06,...,53.458386,2.245848e+07,2.256368e+07,2.277992e+07,2.431527e+07,-2.116684e+08,2.316037e+07,22691850.0,2.222333e+07,1666.680241
3958,16.924705,19.480131,22320000.0,22430800.0,22571300.0,22662550.0,2.482275e+07,48.006214,2346.997775,1.477323e+06,...,53.335951,2.243999e+07,2.253446e+07,2.274402e+07,2.425257e+07,-1.222620e+08,2.312610e+07,22662550.0,2.219900e+07,1587.817986
3959,16.951179,19.480131,23004000.0,22496800.0,22612900.0,22660250.0,2.472663e+07,51.684312,2391.934249,1.478998e+06,...,43.927284,2.262799e+07,2.261983e+07,2.276878e+07,2.421164e+07,1.172543e+08,2.311609e+07,22660250.0,2.220441e+07,1675.061266
3960,16.941832,19.298913,23004000.0,22581400.0,22618800.0,22651250.0,2.462907e+07,47.797063,2377.924759,1.477492e+06,...,45.856043,2.268200e+07,2.265077e+07,2.277080e+07,2.416503e+07,-1.490895e+08,2.308805e+07,22651250.0,2.221445e+07,1505.954700


# 표준화했던 값을 원래 값으로 되돌림

In [19]:
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean
original.shape

(763, 1)

In [20]:
index = df.shape[0] - original.shape[0]
index

3199

In [21]:
y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
rmse

2.9325077649954987

In [22]:
def measure_accuarcy(predict, real = dataset_test.values[:,0], n_days=1):
    predict_result = []
    real_result = []
    count = 0
    result = 0
    correct_up = 0
    correct_down = 0
    real_up = 0
    real_down = 0
    
    n_predicting_days = predict.shape[0]-n_days
    
    for i in range(0,n_predicting_days):
        if predict[i] > predict[i+n_days]:
            result = 1
        else:
            result = 0
        predict_result.append(result)
    
    for i in range(0,n_predicting_days):
        if real[i] > real[i+n_days]:
            result = 1
        else:
            result = 0
        real_result.append(result)
        
    for i in range(0,n_predicting_days):
        if real_result[i]==1:
            real_up += 1
            if predict_result[i] ==1:
                correct_up += 1
        elif real_result[i]==0:
            if predict_result[i] == 0:
                correct_down += 1
                
    count = correct_up + correct_down
    
    return (count/n_predicting_days,correct_up,correct_down)

In [23]:
res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))

In [24]:
def measure_diff(predict, real = dataset_test.values[:,0], n_days=1):
    
    predict_result = []
    real_result = []
    result = 0
    predict_diff = 0
    
    n_predicting_days = predict.shape[0]-n_days
    
    for i in range(0,n_predicting_days):
        if predict[i] > predict[i+n_days]:
            result = 1
        else:
            result = 0
        predict_result.append(result)
    
    for i in range(0,n_predicting_days):
        if real[i] > real[i+n_days]:
            result = 1
        else:
            result = 0
        real_result.append(result)
        
    for i in range(0,n_predicting_days):
        if real_result[i]==predict_result[i]:
            predict_diff += abs(predict[i]-real[i])
        else:
            predict_diff -= abs(predict[i]-real[i])
    
    return predict_diff

In [25]:
measure_diff(original, n_days = 15)

array([17.290388], dtype=float32)

In [26]:
model2 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model2.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model2.add(LSTM(100, return_sequences=True))
model2.add(Dropout(0.8))


model2.add(LSTM(100, return_sequences=True))
model2.add(Dropout(0.8))

#model2.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model2.add(LSTM(units = 100, return_sequences = True))
model2.add(Dropout(0.8))

# Adding a third LSTM layer and some Dropout regularisation

model2.add(LSTM(units = 60, return_sequences = True))
model2.add(Dropout(0.8))

model2.add(LSTM(units = 60, return_sequences = True))
model2.add(Dropout(0.8))


model2.add(LSTM(units = 60, return_sequences = True))
model2.add(Dropout(0.8))

# Adding a fourth LSTM layer and some Dropout regularisation

model2.add(LSTM(units = 60))
model2.add(Dropout(0.8))


# Adding the output layer i 
 
model2.add(Dense(units = 1))

# Compiling the RNN

model2.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model2.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 12s 19ms/step - loss: 0.2848
Epoch 2/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1652
Epoch 3/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1429
Epoch 4/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1392
Epoch 5/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1213
Epoch 6/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1125
Epoch 7/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1166
Epoch 8/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1082
Epoch 9/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1081
Epoch 10/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1090
Epoch 11/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1060
Epoch 12/80
98/98 [==============================] - 2s 19ms/step - loss: 0.1060
Epoch 13/80
98/98 [=================

In [27]:
predicted_stock_price = model2.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 8ms/step
2.8892639856584084
[28.809021]


In [28]:
model3 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model3.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model3.add(LSTM(100, return_sequences=True))
model3.add(Dropout(0.4))


model3.add(LSTM(100, return_sequences=True))
model3.add(Dropout(0.4))

#model3.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model3.add(LSTM(units = 100, return_sequences = True))
model3.add(Dropout(0.4))

# Adding a third LSTM layer and some Dropout regularisation

model3.add(LSTM(units = 100, return_sequences = True))
model3.add(Dropout(0.4))

model3.add(LSTM(units = 100, return_sequences = True))
model3.add(Dropout(0.4))


model3.add(LSTM(units = 100, return_sequences = True))
model3.add(Dropout(0.4))

# Adding a fourth LSTM layer and some Dropout regularisation

model3.add(LSTM(units = 100))
model3.add(Dropout(0.4))


# Adding the output layer i 
 
model3.add(Dense(units = 1))

# Compiling the RNN

model3.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model3.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 12s 20ms/step - loss: 0.1245
Epoch 2/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0577
Epoch 3/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0466
Epoch 4/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0429
Epoch 5/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0379
Epoch 6/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0339
Epoch 7/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0306
Epoch 8/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0292
Epoch 9/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0316
Epoch 10/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0274
Epoch 11/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0268
Epoch 12/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0257
Epoch 13/80
98/98 [=================

In [29]:
predicted_stock_price = model3.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 8ms/step
2.8358246729206904
[35.618885]


In [30]:
model4 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model4.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model4.add(LSTM(100, return_sequences=True))
model4.add(Dropout(0.8))


model4.add(LSTM(100, return_sequences=True))
model4.add(Dropout(0.8))

#model4.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model4.add(LSTM(units = 100, return_sequences = True))
model4.add(Dropout(0.8))

# Adding a third LSTM layer and some Dropout regularisation

model4.add(LSTM(units = 100, return_sequences = True))
model4.add(Dropout(0.8))

model4.add(LSTM(units = 100, return_sequences = True))
model4.add(Dropout(0.8))


model4.add(LSTM(units = 100, return_sequences = True))
model4.add(Dropout(0.8))

# Adding a fourth LSTM layer and some Dropout regularisation

model4.add(LSTM(units = 100))
model4.add(Dropout(0.8))


# Adding the output layer i 
 
model4.add(Dense(units = 1))

# Compiling the RNN

model4.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model4.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 12s 20ms/step - loss: 0.2473
Epoch 2/80
98/98 [==============================] - 2s 20ms/step - loss: 0.1447
Epoch 3/80
98/98 [==============================] - 2s 20ms/step - loss: 0.1156
Epoch 4/80
98/98 [==============================] - 2s 20ms/step - loss: 0.1009
Epoch 5/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0983
Epoch 6/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0973
Epoch 7/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0970
Epoch 8/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0853
Epoch 9/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0792
Epoch 10/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0845
Epoch 11/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0815
Epoch 12/80
98/98 [==============================] - 2s 20ms/step - loss: 0.0743
Epoch 13/80
98/98 [=================

In [31]:
predicted_stock_price = model4.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 8ms/step
2.918148628651311
[1.2399864]


In [32]:
model5 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model5.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model5.add(LSTM(100, return_sequences=True))
model5.add(Dropout(0.4))

#model5.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model5.add(LSTM(units = 100, return_sequences = True))
model5.add(Dropout(0.4))

# Adding a third LSTM layer and some Dropout regularisation

model5.add(LSTM(units = 60, return_sequences = True))
model5.add(Dropout(0.4))

model5.add(LSTM(units = 60, return_sequences = True))
model5.add(Dropout(0.4))

# Adding a fourth LSTM layer and some Dropout regularisation

model5.add(LSTM(units = 60))
model5.add(Dropout(0.4))

# Adding the output layer i 
 
model5.add(Dense(units = 1))

# Compiling the RNN

model5.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model5.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 9s 15ms/step - loss: 0.1086
Epoch 2/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0534
Epoch 3/80
98/98 [==============================] - 1s 15ms/step - loss: 0.0427
Epoch 4/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0398
Epoch 5/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0356
Epoch 6/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0334
Epoch 7/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0328
Epoch 8/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0308
Epoch 9/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0323
Epoch 10/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0282
Epoch 11/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0304
Epoch 12/80
98/98 [==============================] - 1s 14ms/step - loss: 0.0266
Epoch 13/80
98/98 [==================

In [33]:
predicted_stock_price = model5.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 6ms/step
2.920311683913218
[45.448383]


In [34]:
model6 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model6.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model6.add(LSTM(100, return_sequences=True))
model6.add(Dropout(0.8))

#model6.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model6.add(LSTM(units = 100, return_sequences = True))
model6.add(Dropout(0.8))

# Adding a third LSTM layer and some Dropout regularisation

model6.add(LSTM(units = 60, return_sequences = True))
model6.add(Dropout(0.8))

model6.add(LSTM(units = 60, return_sequences = True))
model6.add(Dropout(0.8))

# Adding a fourth LSTM layer and some Dropout regularisation

model6.add(LSTM(units = 60))
model6.add(Dropout(0.8))

# Adding the output layer i 
 
model6.add(Dense(units = 1))

# Compiling the RNN

model6.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model6.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 9s 14ms/step - loss: 0.2497
Epoch 2/80
98/98 [==============================] - 1s 14ms/step - loss: 0.1542
Epoch 3/80
98/98 [==============================] - 1s 14ms/step - loss: 0.1316
Epoch 4/80
98/98 [==============================] - 1s 15ms/step - loss: 0.1142
Epoch 5/80
98/98 [==============================] - 2s 16ms/step - loss: 0.1127
Epoch 6/80
98/98 [==============================] - 1s 15ms/step - loss: 0.1046
Epoch 7/80
98/98 [==============================] - 1s 15ms/step - loss: 0.1026
Epoch 8/80
98/98 [==============================] - 1s 15ms/step - loss: 0.1045
Epoch 9/80
98/98 [==============================] - 1s 15ms/step - loss: 0.1073
Epoch 10/80
98/98 [==============================] - 1s 14ms/step - loss: 0.1029
Epoch 11/80
98/98 [==============================] - 1s 15ms/step - loss: 0.1044
Epoch 12/80
98/98 [==============================] - 1s 15ms/step - loss: 0.1043
Epoch 13/80
98/98 [==================

In [35]:
predicted_stock_price = model6.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 6ms/step
2.986405803738902
[12.6847515]


In [36]:
model7 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model7.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model7.add(LSTM(100, return_sequences=True))
model7.add(Dropout(0.4))

#model7.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model7.add(LSTM(units = 100, return_sequences = True))
model7.add(Dropout(0.4))

# Adding a third LSTM layer and some Dropout regularisation

model7.add(LSTM(units = 100, return_sequences = True))
model7.add(Dropout(0.4))

model7.add(LSTM(units = 100, return_sequences = True))
model7.add(Dropout(0.4))

# Adding a fourth LSTM layer and some Dropout regularisation

model7.add(LSTM(units = 100))
model7.add(Dropout(0.4))

# Adding the output layer i 
 
model7.add(Dense(units = 1))

# Compiling the RNN

model7.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model7.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 9s 16ms/step - loss: 0.1164
Epoch 2/80
98/98 [==============================] - 2s 15ms/step - loss: 0.0472
Epoch 3/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0404
Epoch 4/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0371
Epoch 5/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0335
Epoch 6/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0336
Epoch 7/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0272
Epoch 8/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0281
Epoch 9/80
98/98 [==============================] - 2s 15ms/step - loss: 0.0240
Epoch 10/80
98/98 [==============================] - 1s 15ms/step - loss: 0.0247
Epoch 11/80
98/98 [==============================] - 2s 15ms/step - loss: 0.0254
Epoch 12/80
98/98 [==============================] - 1s 15ms/step - loss: 0.0217
Epoch 13/80
98/98 [==================

In [37]:
predicted_stock_price = model7.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 6ms/step
2.892589331581533
[8.5868225]


In [38]:
model8 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model8.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model8.add(LSTM(100, return_sequences=True))
model8.add(Dropout(0.8))

#model8.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model8.add(LSTM(units = 100, return_sequences = True))
model8.add(Dropout(0.8))

# Adding a third LSTM layer and some Dropout regularisation

model8.add(LSTM(units = 100, return_sequences = True))
model8.add(Dropout(0.8))

model8.add(LSTM(units = 100, return_sequences = True))
model8.add(Dropout(0.8))

# Adding a fourth LSTM layer and some Dropout regularisation

model8.add(LSTM(units = 100))
model8.add(Dropout(0.8))

# Adding the output layer i 
 
model8.add(Dense(units = 1))

# Compiling the RNN

model8.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model8.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 9s 15ms/step - loss: 0.2089
Epoch 2/80
98/98 [==============================] - 1s 15ms/step - loss: 0.1214
Epoch 3/80
98/98 [==============================] - 1s 15ms/step - loss: 0.1006
Epoch 4/80
98/98 [==============================] - 2s 15ms/step - loss: 0.0985
Epoch 5/80
98/98 [==============================] - 1s 15ms/step - loss: 0.0877
Epoch 6/80
98/98 [==============================] - 1s 15ms/step - loss: 0.0856
Epoch 7/80
98/98 [==============================] - 2s 16ms/step - loss: 0.0772
Epoch 8/80
98/98 [==============================] - 2s 15ms/step - loss: 0.0778
Epoch 9/80
98/98 [==============================] - 1s 15ms/step - loss: 0.0788
Epoch 10/80
98/98 [==============================] - 1s 15ms/step - loss: 0.0749
Epoch 11/80
98/98 [==============================] - 2s 15ms/step - loss: 0.0722
Epoch 12/80
98/98 [==============================] - 2s 15ms/step - loss: 0.0748
Epoch 13/80
98/98 [==================

In [39]:
predicted_stock_price = model8.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 2s 6ms/step
2.9395730932374176
[46.433178]


In [40]:
model9 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model9.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model9.add(LSTM(100, return_sequences=True))
model9.add(Dropout(0.4))


model9.add(LSTM(60, return_sequences=True))
model9.add(Dropout(0.4))

#model9.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model9.add(LSTM(60))
model9.add(Dropout(0.4))

# Adding the output layer i 
 
model9.add(Dense(units = 1))

# Compiling the RNN

model9.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model9.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 6s 10ms/step - loss: 0.0854
Epoch 2/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0481
Epoch 3/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0409
Epoch 4/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0400
Epoch 5/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0334
Epoch 6/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0346
Epoch 7/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0309
Epoch 8/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0302
Epoch 9/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0292
Epoch 10/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0277
Epoch 11/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0276
Epoch 12/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0268
Epoch 13/80
98/98 [==================

In [41]:
predicted_stock_price = model9.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 5ms/step
2.8542923503522846
[9.012686]


In [42]:
model10 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model10.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model10.add(LSTM(100, return_sequences=True))
model10.add(Dropout(0.8))


model10.add(LSTM(60, return_sequences=True))
model10.add(Dropout(0.8))

#model10.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model10.add(LSTM(60))
model10.add(Dropout(0.8))

# Adding the output layer i 
 
model10.add(Dense(units = 1))

# Compiling the RNN

model10.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model10.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 7s 10ms/step - loss: 0.2664
Epoch 2/80
98/98 [==============================] - 1s 10ms/step - loss: 0.1363
Epoch 3/80
98/98 [==============================] - 1s 10ms/step - loss: 0.1243
Epoch 4/80
98/98 [==============================] - 1s 10ms/step - loss: 0.1031
Epoch 5/80
98/98 [==============================] - 1s 10ms/step - loss: 0.1099
Epoch 6/80
98/98 [==============================] - 1s 10ms/step - loss: 0.1064
Epoch 7/80
98/98 [==============================] - 1s 10ms/step - loss: 0.1028
Epoch 8/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0978
Epoch 9/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0971
Epoch 10/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0965
Epoch 11/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0919
Epoch 12/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0910
Epoch 13/80
98/98 [==================

In [43]:
predicted_stock_price = model10.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 5ms/step
2.9463290966548787
[21.344585]


In [44]:
model11 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model11.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model11.add(LSTM(100, return_sequences=True))
model11.add(Dropout(0.4))


model11.add(LSTM(100, return_sequences=True))
model11.add(Dropout(0.4))

#model11.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model11.add(LSTM(100))
model11.add(Dropout(0.4))

# Adding the output layer i 
 
model11.add(Dense(units = 1))

# Compiling the RNN

model11.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model11.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 6s 11ms/step - loss: 0.0829
Epoch 2/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0394
Epoch 3/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0344
Epoch 4/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0322
Epoch 5/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0287
Epoch 6/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0305
Epoch 7/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0252
Epoch 8/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0272
Epoch 9/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0246
Epoch 10/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0221
Epoch 11/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0246
Epoch 12/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0210
Epoch 13/80
98/98 [==================

In [45]:
predicted_stock_price = model11.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 5ms/step
2.935279096884328
[5.754219]


In [46]:
model12 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model12.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model12.add(LSTM(100, return_sequences=True))
model12.add(Dropout(0.8))


model12.add(LSTM(100, return_sequences=True))
model12.add(Dropout(0.8))

#model12.add(RepeatVector(time_step))

# Adding a second LSTM layer and some Dropout regularisation

model12.add(LSTM(100))
model12.add(Dropout(0.8))

# Adding the output layer i 
 
model12.add(Dense(units = 1))

# Compiling the RNN

model12.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model12.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 6s 11ms/step - loss: 0.2126
Epoch 2/80
98/98 [==============================] - 1s 11ms/step - loss: 0.1142
Epoch 3/80
98/98 [==============================] - 1s 10ms/step - loss: 0.1016
Epoch 4/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0921
Epoch 5/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0864
Epoch 6/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0769
Epoch 7/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0776
Epoch 8/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0716
Epoch 9/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0666
Epoch 10/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0702
Epoch 11/80
98/98 [==============================] - 1s 10ms/step - loss: 0.0688
Epoch 12/80
98/98 [==============================] - 1s 11ms/step - loss: 0.0737
Epoch 13/80
98/98 [==================

In [47]:
predicted_stock_price = model12.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 5ms/step
2.992910387300827
[30.489231]


In [48]:
model13 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model13.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model13.add(LSTM(60))
model13.add(Dropout(0.4))


# Adding the output layer i 
 
model13.add(Dense(units = 1))

# Compiling the RNN

model13.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model13.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.0878
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0450
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0386
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0352
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0323
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0312
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0314
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0293
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0283
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0279
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0261
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0262
Epoch 13/80
98/98 [==============================

In [49]:
predicted_stock_price = model13.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.859623818567312
[31.88857]


In [50]:
model14 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model14.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model14.add(LSTM(60))
model14.add(Dropout(0.8))


# Adding the output layer i 
 
model14.add(Dense(units = 1))

# Compiling the RNN

model14.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model14.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 7ms/step - loss: 0.2267
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1178
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1099
Epoch 4/80
98/98 [==============================] - 1s 7ms/step - loss: 0.1085
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1013
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0957
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0940
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0856
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0931
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0917
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0913
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0878
Epoch 13/80
98/98 [==============================

In [51]:
predicted_stock_price = model14.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.971181677179522
[23.836319]


In [52]:
model15 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model15.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model15.add(LSTM(100))
model15.add(Dropout(0.4))


# Adding the output layer i 
 
model15.add(Dense(units = 1))

# Compiling the RNN

model15.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model15.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.0826
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0379
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0334
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0324
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0293
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0274
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0254
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0264
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0236
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0230
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0218
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0213
Epoch 13/80
98/98 [==============================

In [53]:
predicted_stock_price = model15.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.9262576168084125
[26.200932]


In [54]:
model16 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model16.add(LSTM(units = 100, return_sequences = True, input_shape = (X_train.shape[-2:])))

model16.add(LSTM(100))
model16.add(Dropout(0.8))


# Adding the output layer i 
 
model16.add(Dense(units = 1))

# Compiling the RNN

model16.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model16.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.1683
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0889
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0868
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0740
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0751
Epoch 6/80
98/98 [==============================] - 1s 7ms/step - loss: 0.0719
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0716
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0750
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0703
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0652
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0640
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0613
Epoch 13/80
98/98 [==============================

In [55]:
predicted_stock_price = model16.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.9548031040983336
[15.111841]


In [56]:
model17 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model17.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[-2:])))

model17.add(LSTM(60))
model17.add(Dropout(0.8))


# Adding the output layer i 
 
model17.add(Dense(units = 1))

# Compiling the RNN

model17.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model17.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2222
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1153
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1106
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1015
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0939
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0982
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0867
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0938
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0918
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0919
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0859
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0849
Epoch 13/80
98/98 [==============================

In [57]:
predicted_stock_price = model17.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.962828392234523
[26.648994]


In [58]:
model18 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model18.add(LSTM(units = 60, return_sequences = True, input_shape = (X_train.shape[-2:])))

model18.add(LSTM(40))
model18.add(Dropout(0.8))


# Adding the output layer i 
 
model18.add(Dense(units = 1))

# Compiling the RNN

model18.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model18.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2515
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1475
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1345
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1357
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1303
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1241
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1219
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1261
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1163
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1131
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1089
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1148
Epoch 13/80
98/98 [==============================

In [59]:
predicted_stock_price = model18.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
3.010308060730538
[35.57329]


In [60]:
model19 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model19.add(LSTM(units = 40, return_sequences = True, input_shape = (X_train.shape[-2:])))

model19.add(LSTM(60))
model19.add(Dropout(0.8))


# Adding the output layer i 
 
model19.add(Dense(units = 1))

# Compiling the RNN

model19.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model19.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2146
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1174
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1067
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0948
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0915
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0914
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0904
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0960
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0895
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0877
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0890
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0826
Epoch 13/80
98/98 [==============================

In [61]:
predicted_stock_price = model19.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
3.02687430926232
[46.31391]


In [62]:
model20 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model20.add(LSTM(units = 40, return_sequences = True, input_shape = (X_train.shape[-2:])))

model20.add(LSTM(40))
model20.add(Dropout(0.8))


# Adding the output layer i 
 
model20.add(Dense(units = 1))

# Compiling the RNN

model20.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model20.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2901
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1572
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1479
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1303
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1205
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1225
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1220
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1195
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1162
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1180
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1070
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1028
Epoch 13/80
98/98 [==============================

In [63]:
predicted_stock_price = model20.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.9808229776481125
[34.64051]


In [64]:
model21 = Sequential()

#Adding the first LSTM layer and some Dropout regularisation

model21.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[-2:])))

model21.add(LSTM(50))
model21.add(Dropout(0.8))


# Adding the output layer i 
 
model21.add(Dense(units = 1))

# Compiling the RNN

model21.compile(optimizer = 'adam' , loss = 'mean_squared_error' )

# Fitting the RNN to the Training set

model21.fit(X_train, y_train, epochs = 80, batch_size = 32)

Epoch 1/80
98/98 [==============================] - 3s 6ms/step - loss: 0.2228
Epoch 2/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1278
Epoch 3/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1100
Epoch 4/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1034
Epoch 5/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1032
Epoch 6/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1012
Epoch 7/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1004
Epoch 8/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0977
Epoch 9/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0959
Epoch 10/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0953
Epoch 11/80
98/98 [==============================] - 1s 6ms/step - loss: 0.0986
Epoch 12/80
98/98 [==============================] - 1s 6ms/step - loss: 0.1000
Epoch 13/80
98/98 [==============================

In [65]:
predicted_stock_price = model21.predict(X_test)
log = feature.iloc[:feature.shape[0]-X_test.shape[0], 0].values
data_mean = log.mean(axis=0) 
data_std = log.std(axis=0)
original = (predicted_stock_price)*data_std+data_mean

y = feature.iloc[feature.shape[0]-X_test.shape[0]:, 1:2].values
rmse = np.sqrt(np.mean(((original - y) ** 2)))
print(rmse)

res.append(round(measure_accuarcy(original, n_days = 15)[0]  * 100,2))
print(measure_diff(original, n_days = 15))

24/24 [==============================] - 1s 3ms/step
2.9906063610686147
[32.2612]


In [66]:
score = pd.DataFrame(res)
score

,0
0,54.55
1,62.57
2,63.50
3,55.21
4,61.36
5,67.25
6,58.29
7,71.39
8,54.01
9,65.78


In [67]:
score.to_csv('res_5.csv', index=False)